In [167]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
engine = create_engine("postgresql:///kc_housing")
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [168]:
def pullsqldata():
    """This function pulls the necessary columns and rows from the PostGRES DB into a Pandas Dataframe in order 
    to continue with our EDA """
    
    engine = create_engine("postgresql:///kc_housing")
    query = """
                SELECT *
                FROM rpsale AS s
                INNER JOIN resbldg AS b ON CONCAT(s.Major,s.Minor) = CONCAT(b.Major, b.Minor)
                INNER JOIN parcel AS p ON CONCAT(s.Major,s.Minor) = CONCAT(p.Major,p.Minor)
                WHERE EXTRACT(YEAR FROM CAST(documentdate AS DATE)) = 2018
                    AND p.proptype = 'R'
                ;"""
    kc_df = pd.read_sql(sql = query, con = engine)
    return kc_df

In [169]:
df = pullsqldata()

In [4]:
def clean_data_initial(df):
    """ This function cleans the housing data by removing outliers and sale price < 10000
    """
    df_clean = df[(df['saleprice']>10000) & (df['saleprice'] <  (2*df['saleprice'].std())+df['saleprice'].mean())]
    return df_clean

In [5]:
df_clean = clean_data_initial(df)

In [125]:
df_clean_drop = df_clean.drop(['documentdate', 
                               'excisetaxnbr', 
                               'recordingnbr', 
                               'volume',
                               'page',
                               'platnbr',
                               'plattype',
                               'platlot',
                               'platblock',
'sellername',
'buyername',
'streetname',
'streettype',
'directionsuffix',
'zipcode',
'buildingnumber',
'major',
'minor',
'bldggradevar',
'sqfthalffloor',
'sqft2ndfloor',
'sqftupperfloor',
'sqftunfinfull',
'sqftunfinhalf',
'sqfttotbasement',
'sqftfinbasement',
'brickstone',
'viewutilization',
'propname',
'platname',
'platlot',
'platblock',
'range',
'township',
'section',
'quartersection',
'area',
'subarea',
'specarea',
'specsubarea',
'levycode',
'districtname',
'currentzoning',
'topography',
'currentusedesignation',
'salewarning', 
'wetland', 
'stream',
'seismichazard',
'landslidehazard',
'address', 
'airportnoise',
'contamination',
'dnrlease',
 'coalminehazard',
 'criticaldrainage',
 'erosionhazard',
 'landfillbuffer',
 'hundredyrfloodplain',
 'steepslopehazard',
 'speciesofconcern',
 'sensitiveareatract',
 'daylightbasement',
 'fraction',
'directionprefix', 'proptype','unbuildable'], axis=1)

In [ ]:
# strips whitespace from othernuisances column, then converts Y to 1 and N to 0
df_clean['othernuisances'] = [x.strip() for x in df_clean['othernuisances']]
df_clean.replace(('Y', 'N'), (1, 0), inplace=True)


In [77]:
Y = df_clean['saleprice']
X = df_clean.drop(['saleprice'], axis=1)


In [9]:
lr = LinearRegression()

In [107]:
# worked to strip whitespace
# X['othernuisances'] = [x.strip() for x in X['othernuisances']]

# I dont think the below worked
# pd.Series(map(lambda x: dict(Y=1, N=0)[x],
#               X.othernuisances.values.tolist()), X.index)


In [117]:
select = RFE(lr, n_features_to_select=50)
select = select.fit(X, y= Y.values.ravel())
selected_columns = X.columns[select.support_]

In [119]:
selected_columns

Index(['principaluse', 'saleinstrument', 'afforestland', 'afcurrentuseland',
       'afnonprofituse', 'afhistoricproperty', 'bldgnbr', 'nbrlivingunits',
       'stories', 'bldggrade', 'finbasementgrade', 'heatsource',
       'bathhalfcount', 'bath3qtrcount', 'bathfullcount', 'fpsinglestory',
       'fpmultistory', 'pcntnetcondition', 'hbuasifvacant', 'hbuasimproved',
       'watersystem', 'sewersystem', 'restrictiveszshape', 'inadequateparking',
       'mtrainier', 'olympics', 'territorial', 'seattleskyline', 'pugetsound',
       'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'wfntbank',
       'wfntrestrictedaccess', 'wfntaccessrights', 'wfntproximityinfluence',
       'powerlines', 'othernuisances', 'nbrbldgsites', 'adjacentgolffairway',
       'adjacentgreenbelt', 'historicsite', 'nativegrowthprotesmt',
       'easements', 'otherdesignation', 'deedrestrictions',
       'developmentrightspurch', 'waterproblems', 'transpconcurrency',
       'otherproblems'],
      dtype='ob

In [134]:
X_int = sm.add_constant(X)
model = sm.OLS(Y, X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     365.4
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        16:39:31   Log-Likelihood:            -3.9876e+05
No. Observations:               28747   AIC:                         7.977e+05
Df Residuals:                   28663   BIC:                         7.984e+05
Df Model:                          83                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   5.417e+06   2.15e+05     25.168      0.000    4.99e+06    5.84e+06
propertytype           -1851.1482    233.065     -7.943      0.000   -2307.966   -1394.330
principaluse           -1.035e+05   2.01e+04     -5.158      0.000   -1.43e+05   -6.42e+04
saleinstrument         -1.198e+04    470.436    -25.474      0.000   -1.29e+04   -1.11e+04
afforestland            4.827e+04   9.78e+04      0.493      0.622   -1.44e+05     2.4e+05
afcurrentuseland       -7.967e+04   5.74e+04     -1.389      0.165   -1.92e+05    3.28e+04
afnonprofituse         -1.893e+05   1.48e+05     -1.277      0.201    -4.8e+05    1.01e+05
afhistoricproperty     -1.408e+05   2.56e+05     -0.549      0.583   -6.43e+05    3.62e+05
salereason             -4014.9325    440.683     -9.111      0.000   -4878.693   -3151.172
propertyclass           -1.44e+04   6813.183     -2.113      0.035   -2.78e+04   -1041.842
bldgnbr                 1.466e+05   6836.427     21.442      0.000    1.33e+05     1.6e+05
nbrlivingunits          5.177e+04   1.12e+04      4.642      0.000    2.99e+04    7.36e+04
stories                 4.198e+04   5013.945      8.374      0.000    3.22e+04    5.18e+04
bldggrade               1.627e+05   2422.664     67.174      0.000    1.58e+05    1.67e+05
sqft1stfloor             -43.5149      6.860     -6.343      0.000     -56.961     -30.068
sqfttotliving            139.7906      4.686     29.831      0.000     130.606     148.976
finbasementgrade        1171.5333    688.420      1.702      0.089    -177.802    2520.869
sqftgaragebasement       -23.5016     10.722     -2.192      0.028     -44.518      -2.485
sqftgarageattached      -140.1887      8.336    -16.818      0.000    -156.527    -123.850
sqftopenporch            110.2881     14.738      7.483      0.000      81.400     139.176
sqftenclosedporch       -158.4191     37.427     -4.233      0.000    -231.777     -85.061
sqftdeck                   2.7104      1.790      1.514      0.130      -0.798       6.219
heatsystem              2824.5466   1519.283      1.859      0.063    -153.319    5802.412
heatsource             -2981.4984   2737.636     -1.089      0.276   -8347.393    2384.397
bedrooms                -2.68e+04   2188.438    -12.244      0.000   -3.11e+04   -2.25e+04
bathhalfcount          -1.158e+04   3635.531     -3.184      0.001   -1.87e+04   -4450.569
bath3qtrcount           4.088e+04   3518.214     11.620      0.000     3.4e+04    4.78e+04
bathfullcount           2.379e+04   3802.013      6.257      0.000    1.63e+04    3.12e+04
fpsinglestory           7148.6522   3482.061      2.053      0.040     323.650     1.4e+04
fpmultistory           -7951.8731   4868.903     -1.633      0.102   -1.75e+04    1591.403
fpfreestanding         -2.639e+04   6106.526     -4.321      0.000   -3.84e+04  

In [166]:
# sns.heatmap(X.corr())
# X.corr() > .75
## looks for things with a correlation of > 0.5
corr = X.corr() > .55
corr_list = []
for col in corr.columns:
    if corr[col].sum() > 1:
        print(col)
        corr_list.append(col)

bldggrade
sqft1stfloor
sqfttotliving
bedrooms
bathfullcount
fpmultistory
fpadditional
olympics
pugetsound
smalllakerivercreek
wfntlocation
wfntfootage
wfntbank
wfntrestrictedaccess
tidelandshoreland


In [160]:
corr_list

['bldggrade',
 'sqft1stfloor',
 'sqfttotliving',
 'bedrooms',
 'bathfullcount',
 'fpmultistory',
 'fpadditional',
 'yrbuilt',
 'olympics',
 'territorial',
 'pugetsound',
 'smalllakerivercreek',
 'wfntlocation',
 'wfntfootage',
 'wfntbank',
 'wfntrestrictedaccess',
 'tidelandshoreland']

In [165]:
# select = RFE(lr, n_features_to_select=50)
# select = select.fit(X, y= Y.values.ravel())
# selected_columns = X.columns[select.support_]

X_corr_list = X[corr_list]

X_int = sm.add_constant(X_corr_list)
model = sm.OLS(Y, X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     24.11
Date:                Tue, 03 Dec 2019   Prob (F-statistic):           3.44e-11
Time:                        16:52:31   Log-Likelihood:            -4.0911e+05
No. Observations:               28747   AIC:                         8.182e+05
Df Residuals:                   28744   BIC:                         8.183e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.147e+05   2177.072    328.268      0.000     7.1e+05    7.19e+05
wfntlocation -8907.6095   4155.786     -2.143      0.032   -1.71e+04    -762.076
wfntbank       1.01e+05   1.72e+04      5.885      0.000    6.74e+04    1.35e+05
==============================================================================
Omnibus:                     4214.190   Durbin-Watson:                   0.830
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6504.918
Skew:                           1.045   Prob(JB):                         0.00
Kurtosis:                       4.030   Cond. No.                         8.17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""